In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping



In [9]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  2


In [10]:
# データの読み込み
data = pd.read_csv('../data/preprocessed_data.csv')

# 特徴量とターゲット変数の分割
X = data.drop('dengue', axis=1).values
y = data['dengue'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
X_train.shape

(3508, 13)

In [12]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [13]:
with strategy.scope():
    # モデルの定義
    model = Sequential()

    # 入力層および隠れ層
    model.add(Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.001), input_dim=13))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
    model.add(Dropout(0.25))
    
    # 出力層（シグモイド活性化関数を使用）
    model.add(Dense(1, activation='sigmoid'))

    # モデルのコンパイル
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])



In [14]:
# モデルのトレーニング
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
history=model.fit(X_train, y_train, epochs=50,validation_split=0.1, batch_size=32,callbacks=[early_stopping])

Epoch 1/50


2024-10-02 17:57:01.488025: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype int64 and shape [3157]
	 [[{{node Placeholder/_11}}]]
2024-10-02 17:57:01.488453: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64 and shape [3157,13]
	 [[{{node Placeholder/_10}}]]


INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1
96/99 [============================>.] - ETA: 0s - loss: 0.7502 - accuracy: 0.6016

2024-10-02 17:57:04.643547: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype int64 and shape [351]
	 [[{{node Placeholder/_11}}]]
2024-10-02 17:57:04.643885: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype int64 and shape [351]
	 [[{{node Placeholder/_11}}]]


99/99 [==============================] - 3s 14ms/step - loss: 0.7485 - accuracy: 0.6018 - val_loss: 0.7045 - val_accuracy: 0.6268
Epoch 2/50
99/99 [==============================] - 1s 10ms/step - loss: 0.6760 - accuracy: 0.6357 - val_loss: 0.6535 - val_accuracy: 0.6353
Epoch 3/50
99/99 [==============================] - 1s 10ms/step - loss: 0.6431 - accuracy: 0.6487 - val_loss: 0.6337 - val_accuracy: 0.6524
Epoch 4/50
99/99 [==============================] - 1s 11ms/step - loss: 0.6246 - accuracy: 0.6516 - val_loss: 0.6202 - val_accuracy: 0.6496
Epoch 5/50
99/99 [==============================] - 1s 11ms/step - loss: 0.6054 - accuracy: 0.6639 - val_loss: 0.6154 - val_accuracy: 0.6695
Epoch 6/50
99/99 [==============================] - 1s 11ms/step - loss: 0.6006 - accuracy: 0.6623 - val_loss: 0.6109 - val_accuracy: 0.6553
Epoch 7/50
99/99 [==============================] - 1s 11ms/step - loss: 0.5945 - accuracy: 0.6690 - val_loss: 0.6037 - val_accuracy: 0.6382
Epoch 8/50
99/99 [======

In [15]:
# テストデータを使って予測
y_pred = model.predict(X_test)

2024-10-02 17:57:29.306011: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64 and shape [878,13]
	 [[{{node Placeholder/_10}}]]
2024-10-02 17:57:29.306360: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64 and shape [878,13]
	 [[{{node Placeholder/_10}}]]


28/28 [==============================] - 0s 3ms/step
